In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import requests      
import numpy as np   
import pandas as pd  
import time          
from bs4 import BeautifulSoup
import random
import re
import json

In [4]:
# Функция, которая парсит бесплатные прокси.
# На вход получает html
# Возвращает список с прокси. Протокол выбран только http
def get_proxies(html):
    final_proxies_list = []
    soup = BeautifulSoup(html.content, 'html.parser')
    proxies_list = soup.find('tbody').find_all('tr')
    for j in proxies_list:
        proxy_row = j.find_all('td')
        ip = proxy_row[0].text
        port = proxy_row[1].text
        if proxy_row[6].text == 'no':
            protocol = 'http'
            proxy = protocol + '://' + ip + ':' + port
            final_proxies_list.append(proxy)
        else:
            continue
    return final_proxies_list
proxies_list = get_proxies(requests.get('https://free-proxy-list.net/'))

In [5]:
def getPageLinks(page_number):
    # составляем ссылку на страницу поиска
    link_list =[]
    page_link = 'https://simplewine.ru/catalog/vino/filter/color-krasnoe/page{}/'.format(page_number)
    response = requests.get(page_link, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
    soup = BeautifulSoup(response.content, 'html.parser')
    
    obj_all = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['snippet-star', 'snippet-star--big'])
    
    for obj in obj_all:
        way = 'https://simplewine.ru'+obj.attrs['href']
        way = way.replace('/reviews/','')
        link_list.append(way)
    return link_list

In [28]:
wine_list = []
for i in range(1,100):
    wine_list.extend(getPageLinks(i))
    time.sleep(1)

In [ ]:
len(wine_list)

In [9]:
def ch_0(soup):

    chars_0 = {'название':None,
               'цена':None,
               'рейтинг_SW':None,
               'год':None}

    script = soup.find('script', {'data-skip-moving': '1', 'type': 'text/javascript'})
    json_text = script.text.split('window.dataLayer = ', 1)[1].rsplit(';', 1)[0]
    data = json.loads(json_text)
    chars_0['цена'] = float(data[0]['ecommerce']['detail']['products'][0]['price'])
    chars_0['название'] = data[0]['ecommerce']['detail']['products'][0]['name'].split(',')[0]
    chars_0['год'] = float(data[0]['ecommerce']['detail']['products'][0]['variant'])
    chars_0['рейтинг_SW'] =float(data[0]['ecommerce']['detail']['products'][0]['dimension13'])
    return chars_0

In [10]:
def ch_1(soup):
    chars_1 = {'Стилистика вина':None,
     'Дегустационные характеристики':None,
     'Гастрономия':None,
     'Виноградники':None,
     'Способ выдержки':None,
     'Способ производства':None,
     'История этикетки':None,
     'Это интересно':None}

    s_wine_1 = soup.findAll(attrs={'class':'characteristics-description__item'})

    for obj in s_wine_1:

        name_char = obj.find(attrs={'class':'characteristics-description__item-title'})
        name_char = str(name_char).replace('<h4 class="characteristics-description__item-title">\n','').replace('</h4>','').replace(':','').strip()
        if name_char in chars_1:
            val = obj.find(attrs={'class':'characteristics-description__item-text'})
            val = str(val).replace('<div class="characteristics-description__item-text">\n','').replace('</div>','').replace(':','').strip()
            chars_1[name_char] = val
    return chars_1

In [11]:
def ch_2(soup):
    chars_2 = {'регион':None,
           'год':None,
           'крепость':None,
           'потенциал хранения':None,
           'аппелласьон':None,
           'категория':None,
           'декантация':None,
           'выдержка в емкости':None}

    s_wine_2 = soup.find(attrs={'class':'characteristics-params__list'}).findAll(attrs={'characteristics-params__item'})

    for obj in s_wine_2:
        name_char = re.search(r'\b[0-9a-zA-Zа-яёА-ЯЁ% ]*[:]',str(obj)).group().replace(':','').lower()
        if name_char in chars_2:
            val = obj.find('dd')
            val = obj.find('dd').text.strip()
            chars_2[name_char] = val
    return chars_2

In [12]:
def ch_3(soup):
    chars_3 = soup.findAll(attrs={'class':'serving__suitable-title'})
    temp = re.findall(r'[>]\n*[0-9a-zA-Zа-яёА-ЯЁ% ]*[<]',str(chars_3))
    chars_3 = [i.replace('<','').replace('>','').replace('\n','').strip() for i in temp]
    dish = {'подача':chars_3}
    return dish

In [13]:
def ch_4(soup):

    chars_4 = {'сладость':None,
               'кислотность':None,
               'танины':None,
               'тело':None}

    temp = soup.findAll('li', attrs = {'class':'taste__item'})
    for obj in temp:
        item = obj.find('div', class_='taste__title').text.strip().lower()
        if item in chars_4:
            chars_4[item] = len(re.findall(r'dot_active"',str(obj)))
    return chars_4

In [14]:
def ch_5(soup):
    chars_5 = []
    comments = soup.findAll('div', class_='product-review__item')
    for comment in comments:
        res = comment.find('div', class_='product-review__item-comment').text.strip()
        chars_5.append(res)
        com = {'комментарии':chars_5}
    return com

In [29]:
df = pd.DataFrame()
for sim in wine_list:
    response = requests.get(sim, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
    soup = BeautifulSoup(response.content, 'html.parser')
    df_0 = pd.DataFrame([ch_0(soup)])
    df_1 = pd.DataFrame([ch_1(soup)])
    df_2 = pd.DataFrame([ch_2(soup)])
    df_3 = pd.DataFrame([ch_3(soup)])
    df_4 = pd.DataFrame([ch_4(soup)])
    
    sim += '/reviews/'
    response = requests.get(sim, {'http':proxies_list[random.randint(0, len(proxies_list) - 1)]})
    soup = BeautifulSoup(response.content, 'html.parser')
    df_5 = pd.DataFrame([ch_5(soup)])
    temp_df = pd.concat([df_0,df_1,df_2,df_3,df_4,df_5], axis=1)
    df = pd.concat([df, temp_df], axis=0)
    time.sleep(1)

In [120]:
df

,название,цена,рейтинг_SW,год,Стилистика вина,Дегустационные характеристики,Гастрономия,Виноградники,Способ выдержки,Способ производства,...,аппелласьон,категория,декантация,выдержка в емкости,подача,сладость,кислотность,танины,тело,комментарии
0,Вино Герцъ,1522.0,4.7,2018.0,"красное — фруктовое, среднетелое, с частичной ...",Глубокий рубиновый цвет. В аромате раскрываютс...,"Отлично подходит к тушеному мясу, птице и выде...",Располагаются в субрегионе Семигорье. Возраст ...,Часть вина выдерживают в дубовых бочках в тече...,Виноград собирают вручную. Двойной триаж (сорт...,...,NaN,Защищённое географическое указание (ЗГУ),аэрация,Бочка,"['Выдержанный сыр', 'Курица', 'Утка', 'Говядин...",1.0,3.0,3.0,3.0,"['Обалденное вино,мягкое, но с очень насыщенны..."
1,Вино Лефкадия Красное,1490.0,4.5,2019.0,"красное — округлое, насыщенное, выдержанное в ...",Вино глубокого тёмно-рубинового цвета. В насыщ...,"Хорошо сочетается с мясным бургером, говядиной...",Расположены в Крымском районе к юго-западу от ...,12—16 месяцев в бочках из французского дуба.,Ферментация в нейтральных ёмкостях при контрол...,...,ЗГУ Кубань.Крымск,Защищённое географическое указание (ЗГУ),аэрация,Бочка,"['Бургер', 'Говядина', 'Ягненок']",2.0,3.0,3.0,2.0,"['Хоть в характеристиках и написано, что декан..."
2,Вино Carolina Reserva Pinot Noir,1490.0,4.4,2020.0,"красное — фруктовое, среднетелое, с частичной ...",Это светло-рубиновое в цвете вино раскрывается...,"Подойдет к копченому мясу домашней птицы, сала...",Расположены в Долине Лейда всего в 5 км от оке...,Непродолжительное время частично в стальных ем...,"Виноград собирают вручную, четверть урожая фер...",...,Valle de Leyda,Denominacion de Origen (DO),аэрация,Бочка,['Курица'],1.0,4.0,2.0,3.0,"['Прекрасное вино! Красная вишня во вкусе, в к..."
3,Вино Семисам Резерв,1240.0,4.8,2019.0,"красное — округлое, насыщенное, выдержанное в ...",Ярко-рубиновый цвет. В насыщенном аромате раск...,"Отлично сочетается с тушеным мясом, овощами гр...",Располагаются на склонах горного хребта Семиса...,В течение 12 месяцев в бочках из французского ...,Сбор винограда исключительно вручную. Тщательн...,...,ЗГУ Кубань. Анапа,Защищённое географическое указание (ЗГУ),декантация,Бочка,"['Говядина', 'Свинина']",1.0,3.0,3.0,2.0,['Отличное вино!\r\nОбязательно в декантер мин...
4,Вино Riparosso Montepulciano d&apos;Abruzzo,1990.0,4.4,2021.0,"красное — фруктовое, среднетелое, выдержанное ...",Глубокий рубиновый цвет. Многогранный букет с ...,"Прекрасно дополнит блюда как итальянской, так ...",Расположены на склонах в коммуне Контрогуерра....,В течение 6 месяцев в бочках из славонского дуба.,"Ферментация в емкостях из нержавеющей стали, м...",...,Montepulciano d'Abruzzo DOC,Denominazione di Origine Controllata (DOC),аэрация,Бочка,"['Говядина', 'Кролик', 'Курица']",2.0,2.0,3.0,3.0,['Приятное вино с выразительным ароматом и при...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,Вино Gevrey-Chambertin,21490.0,5.0,2018.0,"красное — элегантное, комплексное, выдержанное...",Вино яркого рубинового цвета c легкими террако...,Вино прекрасно сочетается с блюдами из дичи и ...,"Почвы бедные, глинисто-известняковые. Плотност...",Выдержка в течение 14-18 месяцев в дубовых боч...,Сбор винограда осуществляется вручную. Фермент...,...,Gevrey-Chambertin AOC,Appellation d'Origine Controlee (AOC),аэрация,Бочка,['Утка'],1.0,3.0,3.0,0.0,"['', '', '', '']"
2447,Вино Le Grand Noir Malbec,1440.0,4.6,2020.0,"красное — фруктовое, среднетелое, выдержанное ...",Вино глубокого рубиново-красного цвета. В аром...,Можно порекомендовать к говядине и ягненку с п...,Расположены на склонах высотой до 150 м над ур...,Непродолжительное время в нейтральных емкостях...,Ночной сбор винограда. Ферментация в течение 2...,...,Pays d'Oc IGP,Indication Geographique Protegee (IGP),аэрация,Бочка,"['Говядина', 'Ягненок', 'Сыр', 'Утка']",1.0,2.0,3.0,3.0,"['За эти деньги - прекрасное вино для ужина, м..."
2448,Вино Mille e Una Notte,16990.0,4.9,2018.0,"

In [122]:
df.to_csv('./red_wineSW.csv',sep = '\t',index=False)


,название,цена,рейтинг_SW,год,Стилистика вина,Дегустационные характеристики,Гастрономия,Виноградники,Способ выдержки,Способ производства,...,аппелласьон,категория,декантация,выдержка в емкости,подача,сладость,кислотность,танины,тело,комментарии
0,Вино Герцъ,1522.0,4.7,2018.0,"красное — фруктовое, среднетелое, с частичной ...",Глубокий рубиновый цвет. В аромате раскрываютс...,"Отлично подходит к тушеному мясу, птице и выде...",Располагаются в субрегионе Семигорье. Возраст ...,Часть вина выдерживают в дубовых бочках в тече...,Виноград собирают вручную. Двойной триаж (сорт...,...,NaN,Защищённое географическое указание (ЗГУ),аэрация,Бочка,"['Выдержанный сыр', 'Курица', 'Утка', 'Говядин...",1.0,3.0,3.0,3.0,"['Обалденное вино,мягкое, но с очень насыщенны..."
1,Вино Лефкадия Красное,1490.0,4.5,2019.0,"красное — округлое, насыщенное, выдержанное в ...",Вино глубокого тёмно-рубинового цвета. В насыщ...,"Хорошо сочетается с мясным бургером, говядиной...",Расположены в Крымском районе к юго-западу от ...,12—16 месяцев в бочках из французского дуба.,Ферментация в нейтральных ёмкостях при контрол...,...,ЗГУ Кубань.Крымск,Защищённое географическое указание (ЗГУ),аэрация,Бочка,"['Бургер', 'Говядина', 'Ягненок']",2.0,3.0,3.0,2.0,"['Хоть в характеристиках и написано, что декан..."
2,Вино Carolina Reserva Pinot Noir,1490.0,4.4,2020.0,"красное — фруктовое, среднетелое, с частичной ...",Это светло-рубиновое в цвете вино раскрывается...,"Подойдет к копченому мясу домашней птицы, сала...",Расположены в Долине Лейда всего в 5 км от оке...,Непродолжительное время частично в стальных ем...,"Виноград собирают вручную, четверть урожая фер...",...,Valle de Leyda,Denominacion de Origen (DO),аэрация,Бочка,['Курица'],1.0,4.0,2.0,3.0,"['Прекрасное вино! Красная вишня во вкусе, в к..."
3,Вино Семисам Резерв,1240.0,4.8,2019.0,"красное — округлое, насыщенное, выдержанное в ...",Ярко-рубиновый цвет. В насыщенном аромате раск...,"Отлично сочетается с тушеным мясом, овощами гр...",Располагаются на склонах горного хребта Семиса...,В течение 12 месяцев в бочках из французского ...,Сбор винограда исключительно вручную. Тщательн...,...,ЗГУ Кубань. Анапа,Защищённое географическое указание (ЗГУ),декантация,Бочка,"['Говядина', 'Свинина']",1.0,3.0,3.0,2.0,['Отличное вино!\r\nОбязательно в декантер мин...
4,Вино Riparosso Montepulciano d&apos;Abruzzo,1990.0,4.4,2021.0,"красное — фруктовое, среднетелое, выдержанное ...",Глубокий рубиновый цвет. Многогранный букет с ...,"Прекрасно дополнит блюда как итальянской, так ...",Расположены на склонах в коммуне Контрогуерра....,В течение 6 месяцев в бочках из славонского дуба.,"Ферментация в емкостях из нержавеющей стали, м...",...,Montepulciano d'Abruzzo DOC,Denominazione di Origine Controllata (DOC),аэрация,Бочка,"['Говядина', 'Кролик', 'Курица']",2.0,2.0,3.0,3.0,['Приятное вино с выразительным ароматом и при...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,Вино Gevrey-Chambertin,21490.0,5.0,2018.0,"красное — элегантное, комплексное, выдержанное...",Вино яркого рубинового цвета c легкими террако...,Вино прекрасно сочетается с блюдами из дичи и ...,"Почвы бедные, глинисто-известняковые. Плотност...",Выдержка в течение 14-18 месяцев в дубовых боч...,Сбор винограда осуществляется вручную. Фермент...,...,Gevrey-Chambertin AOC,Appellation d'Origine Controlee (AOC),аэрация,Бочка,['Утка'],1.0,3.0,3.0,0.0,"['', '', '', '']"
2447,Вино Le Grand Noir Malbec,1440.0,4.6,2020.0,"красное — фруктовое, среднетелое, выдержанное ...",Вино глубокого рубиново-красного цвета. В аром...,Можно порекомендовать к говядине и ягненку с п...,Расположены на склонах высотой до 150 м над ур...,Непродолжительное время в нейтральных емкостях...,Ночной сбор винограда. Ферментация в течение 2...,...,Pays d'Oc IGP,Indication Geographique Protegee (IGP),аэрация,Бочка,"['Говядина', 'Ягненок', 'Сыр', 'Утка']",1.0,2.0,3.0,3.0,"['За эти деньги - прекрасное вино для ужина, м..."
2448,Вино Mille e Una Notte,16990.0,4.9,2018.0,"